In [1]:
import librosa
import numpy as np
import glob
from os import path
import os
from os.path import isfile, join

from scipy import stats

In [2]:


train_folder_name = 'train_data'
test_folder_name = 'test_files'


# train files:
onlyfiles = os.listdir(train_folder_name)
onlyfiles

['dog', 'down', 'off', 'on', 'yes']

In [3]:
# make train and test signal as array

#list_of_file = glob.glob(train_folder_name + os.pathsep )
list_of_train_file = glob.glob('train_data\\**\\*')
list_of_train = []
for file in list_of_train_file:
    file_part = file.split(os.sep)[1]
    #print(os.sep)
    #print(file_part)
    y, sr = librosa.load(file, sr=None)
    #mfcc = librosa.feature.mfcc(y=y, sr=sr)
    #mfcc_norm = stats.zscore(mfcc)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, n_fft=1024, hop_length= int(sr*0.01))
    mfcc_d = librosa.feature.delta(mfcc)
    mfcc= np.concatenate([mfcc,mfcc_d])
    #normalize frequency by row
    mfcc_norm = stats.zscore(mfcc,axis=1)
    list_of_train.append((file_part,mfcc_norm))


In [4]:
mfcc.shape
#len(list_of_train)

(26, 101)

In [5]:
# test files:
list_of_test = []
list_of_test_file = glob.glob(test_folder_name + '/*')
#for test with true label
#list_of_test_file = glob.glob(test_folder_name + '\\**\\*')
for file in list_of_test_file:
    #true_label = file.split(os.sep)[1]
    y, sr = librosa.load(file, sr=None)
    #mfcc = librosa.feature.mfcc(y=y, sr=sr)
    #mfcc_norm = stats.zscore(mfcc)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, n_fft=1024, hop_length= int(sr*0.01))
    mfcc_d = librosa.feature.delta(mfcc)
    mfcc= np.concatenate([mfcc,mfcc_d])
    mfcc_norm = stats.zscore(mfcc,axis=1)
    #list_of_test.append((true_label,mfcc_norm))
    list_of_test.append(mfcc_norm)
len(list_of_test)

250

In [6]:
#len(list_of_test)
print(np.shape(list_of_test))

(250, 26, 101)


In [7]:
def compute_minimal_dtw_path(dist_matrix):
    dtw_matrix = np.zeros_like(dist_matrix)
    dtw_matrix[0,:] = np.cumsum(dist_matrix[0,:])
    dtw_matrix[:,0] = np.cumsum(dist_matrix[:,0])
    for i in range(1, np.shape(dist_matrix)[0]):
        for j in range(1,np.shape(dist_matrix)[1]):
            dtw_matrix[i,j] = dist_matrix[i,j] + min(dtw_matrix[i-1,j],dtw_matrix[i,j-1],dtw_matrix[i-1,j-1])       
    return dtw_matrix[-1,-1]

In [11]:
from datetime import datetime

start = datetime.now() 

# make prediction for euclidean and DTW
list_of_labels = []
for test_name,test_fitcher  in zip(list_of_test_file, list_of_test):
    min_distance_euclidean = np.inf
    min_distance_dtw       = np.inf
    for label , train_fitcher in list_of_train:
        #create auxilary matrix for euclidean distances
        dist_matrix = np.zeros((test_fitcher.shape[1], test_fitcher.shape[1]))
        for i in range(test_fitcher.shape[1]):
            for j in range(train_fitcher.shape[1]):
                dist_matrix[i,j] = np.linalg.norm(test_fitcher[:, i]-train_fitcher[:, j])

        # euclideam distance with 1NN
        distance_euclidean = sum(dist_matrix.diagonal(0))
        if distance_euclidean < min_distance_euclidean:
            min_distance_euclidean = distance_euclidean
            label_euclidean = label

        # DTW distance with 1NN
        dtw_distance = compute_minimal_dtw_path(dist_matrix)
        if dtw_distance < min_distance_dtw:
            min_distance_dtw = dtw_distance
            label_dtw = label
    list_of_labels.append((path.basename(test_name), label_euclidean, label_dtw))
   

        
#print(list_of_labels)

In [12]:
print("time:", datetime.now() - start)

time: 0:15:01.140240


In [13]:
with open("output.txt", 'wt') as f:
    f.write('\n'.join([' - '.join(item) for item in list_of_labels]))



In [ ]:
#experiment and evaluation with true label

# from datetime import datetime

# start = datetime.now() 

# score_eucl =0 
# score_dtw =0
# c = 0

# # make prediction for euclidean and DTW
# list_of_labels = []
# for test_name, (true_label ,test_fitcher)  in zip(list_of_test_file, list_of_test):
#    # normalized_test = test_fitcher/np.linalg.norm(test_fitcher)
    
#     #normalized_test = normalize(test_fitcher) 
#     #normalize_test = normalize_columns(test_fitcher)
#     normalized_test = test_fitcher
#     #print(normalized_test)
#     min_distance_euclidean = np.inf
#     min_distance_dtw       = np.inf
#     for label , train_fitcher in list_of_train:
#         #normalized_train = train_fitcher/np.linalg.norm(train_fitcher)
#         #normalized_train = normalize(train_fitcher)
#         #normalize_train = normalize_columns(train_fitcher)
#         normalized_train = train_fitcher
#         #create auxilary matrix for euclidean distances
#         dist_matrix = np.zeros((test_fitcher.shape[1], test_fitcher.shape[1]))
#         for i in range(test_fitcher.shape[1]):
#             for j in range(train_fitcher.shape[1]):
#                 dist_matrix[i,j] = np.linalg.norm(normalized_test[:, i]-normalized_train[:, j])

#         # euclideam distance with 1NN
#         distance_euclidean = sum(dist_matrix.diagonal(0))
#         if distance_euclidean < min_distance_euclidean:
#             min_distance_euclidean = distance_euclidean
#             label_euclidean = label

#         # DTW distance with 1NN
#         dtw_distance = compute_minimal_dtw_path(dist_matrix)
#         if dtw_distance < min_distance_dtw:
#             min_distance_dtw = dtw_distance
#             label_dtw = label
#     list_of_labels.append((true_label,path.basename(test_name), label_euclidean, label_dtw))
#     c += 1
#     if true_label == label_euclidean :
#         score_eucl +=1
#     if true_label == label_dtw :
#         score_dtw +=1
        
# print(score_eucl,score_dtw)

# print("time:", datetime.now() - start)
        
# #print(list_of_labels)
# print(c)
# print(score_eucl/c * 100)
# print(score_dtw/c*100)

In [ ]:
dist_matrix

In [14]:
print('\n'.join([' - '.join(item) for item in list_of_labels]))

0bzvmwox_135c6841_cwmzyh8l.wav - dog - off
0p6f9oym_37fc5d97_p3yx1jqz.wav - off - dog
0quzgaxm_f0ae7203_0tipiwq9.wav - yes - yes
0s7rm82v_8ec6dab6_6qikrrgq.wav - dog - off
0xxii25g_1528225c_zxds31hu.wav - dog - yes
12p6k025_aa48c94a_fiqvuswp.wav - down - down
19m5bbib_24ad3ebe_qfwxzhw5.wav - yes - off
19rldkhz_91223b83_ehzrsz5i.wav - dog - dog
1e7d5r0n_d5ca80c6_pts7hcd3.wav - dog - dog
1p0943j8_563aa4e6_37y0jw0c.wav - dog - dog
1s2v5nr1_9a356ab9_mfj29guq.wav - down - on
1t5vfa33_3f2b358d_6vvy9pxw.wav - dog - dog
1vl0hfs3_7257420c_o12tca6n.wav - on - on
28b17htj_1093c8e7_qwd3ocn4.wav - down - yes
2dyldzeq_aef8dcf5_lcp5bgcy.wav - yes - yes
2hjtahcb_5c8af87a_icd1v70z.wav - off - down
2ifnl7dh_692a88e6_dn25w64z.wav - yes - on
2mk1843a_43fc47a7_pmtyff9h.wav - down - dog
2s8px43j_1538beb7_3pwxtuml.wav - off - yes
2tgg79x9_563aa4e6_kw3opexz.wav - off - off
2w30m4kf_3efef882_7smrzizn.wav - on - down
2ysv1nou_a8e25ebb_yhrwjpcl.wav - yes - yes
316tzlk9_e1469561_lo2c5evf.wav - yes - yes
32t812yv_

In [ ]:

# test_name, test_fitcher = list_of_test_file[0], list_of_test[0]
# normalized_v = test_fitcher/np.linalg.norm(test_fitcher)
# print("vector:",test_fitcher)
# print("normalize",normalized_v)

In [ ]:
#51
#47.9
def normalize(p):
    return (p-p.mean())/p.std()

def normalize(p):
    return stats.zscore(p)

def normalize_columns(m):
    return (m-np.average(m,axis=0))/np.std(m,axis=0)